In [1]:
import requests, re
import pandas as pd
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from Market_API import Market

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:134.0) Gecko/20100101 Firefox/134.0'
}

In [3]:
stock_url = 'https://rahavard365.com/api/v2/market-data/stocks?last_trade=last_trading_day'
stock_res = requests.get(stock_url, headers=headers)
stock_res = stock_res.json()['data']

In [4]:
fund_url = 'https://rahavard365.com/api/v2/market-data/etf-funds?last_trade=last_trading_day&fund_type=1'
fund_res = requests.get(fund_url, headers=headers)
fund_res = fund_res.json()['data']

In [5]:
closed_url = 'https://rahavard365.com/api/v2/market-data/closed-assets'
closed_res = requests.get(closed_url, headers=headers)
closed_res = closed_res.json()['data']['closed_asset_list']

In [6]:
stock_df = pd.DataFrame(stock_res)
fund_df = pd.DataFrame(fund_res)
closed_df = pd.DataFrame(closed_res)

In [7]:
closed_df

,asset_id,name,title,exchange_id,category,reason_of_close,closed_day_count,last_trade_date_time,close_price,company_last_value,company_last_value_percent,slug
0,336,شکبیر,پتروشیمی امیرکبیر (سهامی عام),2,,ممنوع - متوقف - توقف به دلایل دیگر,55,2024-12-08T12:03:03.023+03:30,67800,244080000000000,0.108157,شکبیر
1,131,چکارم,کارتن مشهد,2,,ممنوع - متوقف - توقف به دلایل دیگر,2392,2018-07-16T10:42:08+04:30,2399,35985000000,0.000016,چکارم
2,101,کایگچ,ایران گچ,2,,ممنوع - متوقف - به دلیل برگزاری مجمع فوق العاده,2827,2017-05-07T09:19:28.273+04:30,18500,277500000000,0.000123,کایگچ
3,15701,وساربیل,سر. استان اردبیل,1,,ممنوع - متوقف - به دلیل عدم رعایت الزمات دستور...,1356,2021-05-17T11:24:02.01+04:30,1280,80905984000000,0.035851,وساربیل
4,652,خبازرس,بازرسی مهندسی و صنعتی ایران,2,,ممنوع - متوقف - توقف به دلایل دیگر,6,2025-01-26T12:27:08.775+03:30,11370,3411000000000,0.001511,خبازرس
...,...,...,...,...,...,...,...,...,...,...,...,...
116,598,توسعه,اعتباری توسعه (سهامی عام),2,,ممنوع - متوقف - به دلیل عدم رعایت الزامات دستو...,1757,2020-04-11T12:23:29+04:30,2717,5434000000000,0.002408,توسعه
117,224,قشکر,شکر شاهرود,1,,ممنوع - متوقف - به دلیل مجمع عادی سالیانه,5,2025-01-27T12:29:21.404+03:30,2734,6725894262000,0.002980,قشکر
118,28105,حیات,بیمه زندگی مفید,2,,ممنوع - متوقف - توقف به دلایل دیگر,304,2024-04-03T15:53:13.191+03:30,1000,0,0.000000,حیات
119,33196,ولاعتماد,واسپاری عصر اعتماد,2,,ممنوع - متوقف - توقف به دلایل دیگر,159,2024-08-26T16:09:42.824+03:30,1000,0,0.000000,ولاعتماد


In [8]:
total_symbols = pd.concat([stock_df, closed_df, fund_df])

In [9]:
market = Market()

In [10]:
stock_market_watch = market.get_stock_market_watch()

In [11]:
stock_market_watch['symbol'] = stock_market_watch['symbol'].apply(market.characters_modifier)
total_symbols['name'] = total_symbols['name'].apply(market.characters_modifier)

In [14]:
not_found_symbols = total_symbols[~total_symbols['name'].isin(list(stock_market_watch['symbol'].str.strip()))]

In [15]:
driver = Firefox()

thershold = 5
while thershold:
    try:
        driver.get('https://www.tsetmc.com/MarketOverall')
        driver.find_element(By.XPATH, '//*[@id="search"]').click()
        break
    except:
        thershold -= 1
        time.sleep(1)

pattern = r'href="/instInfo/(\d+)"'

data = []
for symbol in not_found_symbols['name'].unique():

    thershold = 3
    while thershold:
        driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/input').clear()
        driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/input').send_keys(str(symbol))
        time.sleep(.7)
        # driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/input').send_keys(str(symbol)[-1])
        # time.sleep(.1)

        try:
            # result rows
            result_box = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div/div[2]/div/div/div/div[1]/div[2]/div[3]/div[2]/div/div')
            result_rows = result_box.find_elements(By.CLASS_NAME, 'ag-row')
            if not len(result_rows):
                break

            for row in result_rows:
                result_title = row.find_element(By.TAG_NAME, 'div')
                result_symbol = re.search(r'(.*)-(.*)', result_title.text.strip())[1]

                if market.characters_modifier(result_symbol.strip()) == market.characters_modifier(symbol.strip()):
                    href = result_title.find_element(By.TAG_NAME, 'a').get_attribute('outerHTML')
                    id = re.search(pattern, href)[1]
                    data.append([symbol, id])
                    # with open('data.txt', 'a', encoding='utf8') as f:
                    #     f.write(str(id).strip() + ';' + symbol.strip() + '\n')
                    thershold = 0
                    break
        except:
            thershold -= 1
            time.sleep(.1)
            if thershold == 0:
                with open('not_found.txt', 'a', encoding='utf8') as f:
                    f.write(str(symbol) + '\n')
            continue

In [32]:
stock_market_watch = stock_market_watch[['symbol', 'id']]
not_found_df = pd.DataFrame(data, columns=['symbol', 'id'])

firm_ids_df = pd.concat([stock_market_watch, not_found_df])

firm_ids_df = firm_ids_df.drop_duplicates(subset='id')

In [33]:
firm_ids_df

,symbol,id
0,وسپهر,114312662654155
1,شصدف,204092872752957
2,فسا,318005355896147
3,فرآور,408934423224097
4,آکام,490987973229371
...,...,...
99,قشکر,35964395659427029
100,حیات,7826686478591760
101,ولاعتماد,68261410734526618
102,حکمت,7881099657101647


In [ ]:
url = 'https://cdn.tsetmc.com/api/Instrument/GetInstrumentIdentity/' + str(id)

i = 1
firms_info_list = []
for id in firm_ids_df.id.unique():
    res = requests.get(url, headers=headers).json()['instrumentIdentity']

    code = res['instrumentID']
    symbol = res['lVal18AFC']
    name = res['lVal30']
    market_name = res['cgrValCotTitle']
    market_code = res['cComVal']

    industry_dict = res['sector']
    industry_name = industry_dict['lSecVal']
    industry_code = industry_dict['cSecVal']

    sub_industry_dict = res['subSector']
    sub_industry_name = sub_industry_dict['lSoSecVal']
    sub_industry_code = sub_industry_dict['cSoSecVal']
    
    firms_info_list.append([id, code, symbol, name, market_name, market_code, industry_name, industry_code, sub_industry_name, sub_industry_code])
    print(i, id)
    i+=1
    

1 114312662654155
2 204092872752957
3 318005355896147
4 408934423224097
5 490987973229371
6 589599697502308
7 611986653700161
8 655060129740445
9 778253364357513
10 793710053482057
11 831325835570803
12 917857106093847
13 971068957336171
14 1050751214677134
15 1241998328504490
16 1301069819790264
17 1358190916156744
18 1438514795814416
19 1625149423498289
20 1822787329898392
21 1825060609227181
22 2019449432639594
23 2161110547458064
24 2254054929817435
25 2318736941376687
26 2328862017676109
27 2400322364771558
28 2434703913394836
29 2589887561569709
30 2696258159590598
31 2944500421562364
32 2992846609726856
33 3149396562827132
34 3173544097113770
35 3407806799514469
36 3492952121304423
37 3493306453706327
38 3542690854557886
39 3623921205367364
40 3654864906585643
41 3722699128879020
42 3839324986781871
43 3846143218462419
44 3863538898378476
45 3927545069816217
46 3955332316338258
47 4159532151694984
48 4170494561477386
49 4216536645718658
50 4247709727327181
51 4267564158935326
52

In [ ]:
pd.DataFrame(data, columns=['id', 'code', 'symbol', 'name', 'market_name', 'market_code', 'industry_name', 'industry_code', 'sub_industry_name', 'sub_industry_code'])